In [1]:
from mindspore import context
from mindsponge import Sponge
from mindsponge import Molecule
from mindsponge import ForceField
from mindsponge import DynamicUpdater
from mindsponge.control import VelocityVerlet, Langevin
from mindsponge.callback import WriteH5MD, RunInfo

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

## 使用力场模板

MindSponge中支持yaml格式的力场参数信息配置，其中大体分为`模板`和`参数`两个大类。`模板`中主要存储系统中的原子名称、原子类型、原子质量、原子序数、成键关系等基本信息，也是一般都不需要去改动的信息。`参数`中顾名思义就是存储一些力场参数，比如键长相互作用$E_b=k_b(l-l_0)^2$中的$(k_b,l_0)$就是一组力场参数。不同的原子组合形成的键、角度、二面角还有其他的一些远程相互作用等，都会对应于不同的参数，在不同的力场中千差万别。

比如这是一个三点水的模板配置信息：

```yaml
template:
  WAT:
    atom_name: [O, H1, H2]
    atom_type: [OW, HW, HW]
    atom_mass: [16.00, 1.008, 1.008]
    atomic_number: [8, 1, 1]
    bond:
    - [0, 1]
    - [0, 2]
    head_atom: null
    tail_atom: null
```

以及一个三点水的力场参数信息：

```yaml
template:
  base: water.tip3p.yaml
parameters:
  bond_energy:
    length_unit: nm
    energy_unit: kj/mol
    parameter_names:
      atoms: [bond_length, force_constant]
    parameters:
      OW-HW: [0.09572, 502416]
  angle_energy:
    length_unit: nm
    energy_unit: kj/mol
    parameter_names:
      atoms: [bond_angle, force_constant]
    parameters:
      HW-OW-HW: [1.824218, 628.02]
  coulomb_energy:
    length_unit: nm
    energy_unit: kj/mol
  vdw_energy:
    length_unit: nm
    energy_unit: kj/mol
    parameter_names:
      atoms: [sigma, epsilon]
    parameters:
      OW: [0.315061, 0.636386]
      HW: [0.0, 0.0]
```

用户可以根据不同的力场自定义一个这样的yaml文件，然后就可以在MindSponge平台上非常方便的进行分子动力学模拟。

In [3]:
system = Molecule(template='water.tip3p.yaml')

In [4]:
potential = ForceField(system, parameters='TIP3P')

In [5]:
opt = DynamicUpdater(system,
                     integrator=VelocityVerlet(system),
                     thermostat=Langevin(system, 300),
                     time_step=1e-3)

In [6]:
md = Sponge(system, potential, opt)

In [7]:
run_info = RunInfo(10)
cb_h5md = WriteH5MD(system, 'tutorial_b02.h5md', save_freq=10, write_velocity=True, write_force=True)

In [8]:
md.run(1000, callbacks=[run_info, cb_h5md])

Step: 0, E_pot: -321.2302, E_kin: 0.0, E_tot: -321.2302, Temperature: 0.0
Step: 10, E_pot: -338.49777, E_kin: 1.5871477, E_tot: -336.9106, Temperature: 63.629997
Step: 20, E_pot: -353.41055, E_kin: 17.06509, E_tot: -336.34546, Temperature: 684.1529
Step: 30, E_pot: -339.99286, E_kin: 27.341263, E_tot: -312.6516, Temperature: 1096.1327
Step: 40, E_pot: -326.36465, E_kin: 16.677002, E_tot: -309.68765, Temperature: 668.5941
Step: 50, E_pot: -331.90106, E_kin: 1.9247835, E_tot: -329.9763, Temperature: 77.16608
Step: 60, E_pot: -351.152, E_kin: 9.303469, E_tot: -341.84854, Temperature: 372.98337
Step: 70, E_pot: -348.51932, E_kin: 29.548454, E_tot: -318.97086, Temperature: 1184.6207
Step: 80, E_pot: -325.14798, E_kin: 22.25408, E_tot: -302.8939, Temperature: 892.18353
Step: 90, E_pot: -329.14166, E_kin: 3.141025, E_tot: -326.00064, Temperature: 125.92617
Step: 100, E_pot: -349.26553, E_kin: 7.8872676, E_tot: -341.37827, Temperature: 316.20676
Step: 110, E_pot: -349.02707, E_kin: 24.545055, 

![](../../docs/tutorial_b02.gif)